In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project\\reseasrch'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.DataScienceProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.DataScienceProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH, schema_filepath= SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import urllib.request as request
from src.DataScienceProject import logger
import zipfile

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url =self.config.source_url, filename=self.config.local_data_file)
            logger.info(f"File : {filename} baixado com sucesso: \n{headers}")
        else:
            logger.info(f"Arquivo já existe em : {self.config.local_data_file}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Arquivo extraído com sucesso em: {unzip_path}")

In [9]:
try:
    config_manager = ConfigurationManager()
    config_ingestion = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=config_ingestion)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)

[2026-02-04 08:01:46,086: INFO: common: YAML file: config\config.yaml carregado com sucesso:]
[2026-02-04 08:01:46,089: INFO: common: YAML file: params.yaml carregado com sucesso:]
[2026-02-04 08:01:46,093: INFO: common: YAML file: schema.yaml carregado com sucesso:]
[2026-02-04 08:01:46,093: INFO: common: Diretório criado em: artifacts:]
[2026-02-04 08:01:46,096: INFO: common: Diretório criado em: artifacts/data_ingestion:]
[2026-02-04 08:01:46,850: INFO: 1356929599: File : artifacts/data_ingestion/data.zip baixado com sucesso: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 98EA:09D8:3B5701:714CA6:69832719
Accept-Ranges: bytes
Date: Wed